In [ ]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

In [ ]:
using LinearAlgebra
using ForwardDiff

In [ ]:
function hat(v)
    return [0 -v[3] v[2];
            v[3] 0 -v[1];
            -v[2] v[1] 0]
end

In [ ]:
function L(q)
    s = q[1]
    v = q[2:4]
    L = [s    -v';
         v  s*I+hat(v)]
    return L
end

In [ ]:
function R(q)
    s = q[1]
    v = q[2:4]
    R = [s    -v';
         v  s*I-hat(v)]
    return R
end

In [ ]:
T = Diagonal([1; -ones(3)])
H = [zeros(1,3); I];

In [ ]:
function G(q)
    G = L(q)*H
end

function Q(q)
    return H'*(R(q)'*L(q))*H
end

In [ ]:
J = Diagonal([1; 2; 3])
h = 0.1

In [ ]:
#initial conditions
Q0 = Array(I(3))
q0 = [1; 0; 0; 0]
ω0 = randn(3)
x0 = [vec(Q0); ω0]
x0q = [q0; ω0]

In [ ]:
#dynamics
function dynamics(x)
    Q = reshape(x[1:9],3,3)
    ω = x[10:12]
    
    Q̇ = Q*hat(ω)
    ω̇ = -J\(hat(ω)*J*ω)

    ẋ = [vec(Q̇); ω̇]
end

In [ ]:
function rkstep(x)
    f1 = dynamics(x)
    f2 = dynamics(x + 0.5*h*f1)
    f3 = dynamics(x + 0.5*h*f2)
    f4 = dynamics(x + h*f3)
    xn = x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
    return xn
end

In [ ]:
xk = x0
for k = 1:10000
    xk = rkstep(xk)
end

In [ ]:
Qk = reshape(xk[1:9],3,3)

In [ ]:
Qk'*Qk

In [ ]:
#quaternion dynamics
function qdynamics(x)
    q = x[1:4]
    ω = x[5:7]
    
    q̇ = 0.5*L(q)*H*ω
    ω̇ = -J\(hat(ω)*J*ω)

    ẋ = [q̇; ω̇]
end

In [ ]:
function qrkstep(x)
    f1 = qdynamics(x)
    f2 = qdynamics(x + 0.5*h*f1)
    f3 = qdynamics(x + 0.5*h*f2)
    f4 = qdynamics(x + h*f3)
    xn = x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
    xn[1:4] .= xn[1:4]./norm(xn[1:4])
    return xn
end

In [ ]:
xkq = x0q
for k = 1:10000
    xkq = qrkstep(xkq)
end

In [ ]:
qk = xkq[1:4]

In [ ]:
norm(qk)

In [ ]:
Q(qk)'*Q(qk)